In [1]:
import numpy as np
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate

2024-09-15 12:23:28.226027: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 12:23:28.230301: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 12:23:28.273366: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 12:23:28.315827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 12:23:28.351035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

2024-09-15 12:23:29.153102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Exemple de séries temporelles pour plusieurs joueurs
with open('time_on_ice_data.json', 'r') as f:
    time_on_ice = json.load(f)

# Trouver la longueur maximale des séries temporelles pour faire du padding
max_length = max([len(player['past']) for player in time_on_ice.values()])

# Separer en training/testing

# Convertir le dictionnaire time_on_ice en une liste de tuples (clé, valeur)
time_on_ice_items = list(time_on_ice.items())

# Séparer en ensemble d'entraînement et de test (80% training, 20% testing)
train_data, test_data = train_test_split(time_on_ice_items, test_size=0.01, random_state=42)

# Reconvertir les ensembles en dictionnaires
train_dict = dict(train_data)
test_dict = dict(test_data)

# Afficher la taille de chaque ensemble
print(f"Taille de l'ensemble d'entraînement : {len(train_dict)}")
print(f"Taille de l'ensemble de test : {len(test_dict)}")
print(time_on_ice)

Taille de l'ensemble d'entraînement : 148
Taille de l'ensemble de test : 2
{'player1': {'past': [13, 13], 'target': 20, 'age': 34, 'height': 174}, 'player2': {'past': [2, 10, 7, 14, 13, 16, 7, 9], 'target': 7, 'age': 38, 'height': 174}, 'player3': {'past': [1, 16, 13, 5, 1, 19, 6], 'target': 8, 'age': 22, 'height': 174}, 'player4': {'past': [6, 3, 10, 19], 'target': 4, 'age': 25, 'height': 183}, 'player5': {'past': [3, 1, 16, 18, 16, 16, 16, 19, 4], 'target': 16, 'age': 24, 'height': 177}, 'player6': {'past': [13, 12, 9, 17, 3, 16, 14], 'target': 14, 'age': 28, 'height': 190}, 'player7': {'past': [1, 9, 15, 11, 5], 'target': 12, 'age': 38, 'height': 192}, 'player8': {'past': [18, 13, 7, 5, 6, 9, 10, 6, 19, 12], 'target': 12, 'age': 31, 'height': 183}, 'player9': {'past': [4, 14], 'target': 16, 'age': 26, 'height': 177}, 'player10': {'past': [3, 8, 3, 18, 13, 12, 3, 16, 18], 'target': 17, 'age': 32, 'height': 195}, 'player11': {'past': [6, 2, 4, 9], 'target': 1, 'age': 26, 'height': 191

In [3]:
# Extraire les cibles et les variables fixes (âge, grandeur)
targets = np.array([player['target'] for player in train_dict.values()])
ages = np.array([player['age'] for player in train_dict.values()])
heights = np.array([player['height'] for player in train_dict.values()])

targets

array([20, 15, 18, 17, 12,  8, 17,  6, 13,  8, 14, 17, 11, 16,  7, 13,
       10,  7,  2,  5,  7,  1, 14, 19,  9,  5, 18, 18,  6,  6, 17, 15,
        8, 11,  2,  9, 16, 13, 19, 10, 19,  7,  1, 12, 12, 12, 17,  9,
       16, 17, 15, 20, 12,  7, 16, 17, 18,  8, 12, 10,  5, 11,  8, 15,
       15, 14, 15, 13,  3, 19,  5,  7, 14,  4, 11, 10, 14,  3, 12, 12,
       14,  5,  1,  5,  7, 12, 16,  3, 12, 13, 12, 16, 12,  9, 16,  9,
        5, 19,  4,  6, 12,  7, 12, 12,  4, 18,  8, 14,  2, 14, 10,  3,
        8, 17, 19,  5,  7, 12,  9,  8, 11, 11,  4,  6,  9, 20, 16, 11,
       15,  3,  4, 15,  7, 14,  9,  9,  4, 17,  4, 15, 15,  5,  7, 10,
       13, 10, 15,  7])

In [4]:
# Padding des séries temporelles pour qu'elles aient toutes la même longueur
padded_series = [
    [0] * (max_length - len(player['past'])) + player['past']
    for player in train_dict.values()
]

print(padded_series)

# Transformer les données en matrices (entrée pour le LSTM)
X_time_series = np.array(padded_series).reshape(len(train_dict), max_length, 1)
X_fixed = np.column_stack((ages, heights))  # Variables fixes

[[0, 16, 18, 10, 4, 13, 12, 9, 6, 1], [0, 0, 0, 18, 19, 5, 8, 5, 12, 9], [0, 0, 0, 0, 0, 0, 0, 0, 16, 7], [0, 12, 14, 1, 13, 12, 11, 10, 16, 19], [0, 0, 0, 4, 17, 15, 5, 7, 17, 15], [1, 10, 11, 1, 3, 5, 10, 1, 8, 8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 6], [0, 0, 0, 0, 0, 0, 0, 1, 2, 15], [17, 7, 11, 18, 12, 5, 5, 2, 10, 1], [0, 0, 0, 8, 16, 9, 1, 19, 14, 15], [0, 0, 0, 8, 18, 12, 7, 5, 10, 9], [0, 3, 8, 3, 18, 13, 12, 3, 16, 18], [0, 0, 0, 0, 0, 6, 18, 19, 5, 13], [0, 0, 0, 0, 0, 0, 0, 0, 18, 9], [0, 0, 0, 0, 0, 0, 2, 13, 4, 14], [0, 0, 0, 0, 0, 0, 0, 0, 6, 16], [0, 0, 0, 0, 0, 0, 10, 15, 13, 19], [0, 0, 0, 0, 0, 0, 0, 7, 7, 13], [0, 0, 0, 0, 0, 0, 2, 1, 17, 16], [0, 0, 0, 0, 0, 8, 10, 11, 18, 19], [0, 13, 15, 5, 6, 1, 10, 1, 2, 4], [14, 15, 5, 6, 5, 16, 3, 5, 8, 11], [0, 0, 0, 0, 0, 0, 7, 16, 11, 12], [0, 0, 0, 0, 0, 10, 3, 12, 19, 12], [0, 0, 0, 0, 0, 0, 0, 0, 0, 11], [0, 0, 0, 0, 0, 0, 13, 11, 16, 6], [0, 0, 0, 0, 0, 0, 0, 0, 0, 7], [0, 0, 1, 18, 7, 12, 6, 6, 6, 18], [0, 0, 0, 0, 0, 0, 0, 

In [35]:
# Définir les dimensions
num_players = len(train_dict)  # Nombre de joueurs (3 dans cet exemple)
num_fixed_features = X_fixed.shape[1]  # Nombre de variables fixes (2 ici: âge, grandeur)

# Définir les entrées pour le modèle LSTM
input_time_series = Input(shape=(max_length, 1), name="time_series_input")
input_fixed = Input(shape=(num_fixed_features,), name="fixed_input")

In [36]:
# LSTM pour les séries temporelles
lstm_output = LSTM(units=50, activation='relu')(input_time_series)

# Combiner la sortie du LSTM avec les variables fixes
combined = Concatenate()([lstm_output, input_fixed])

# Ajouter une couche dense pour la prédiction finale
output = Dense(1)(combined)

In [37]:
# Définir et compiler le modèle
model = Model(inputs=[input_time_series, input_fixed], outputs=output)
model.compile(optimizer='adam', loss='mse')

In [38]:
# Afficher le résumé du modèle
model.summary()

Model: "functional_1"
Model: "functional_1"


┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)          ┃ Output Shape        ┃     Param # ┃ Connected to        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ time_series_input     │ (None, 10, 1)       │           0 │ -                   │
│ (InputLayer)          │                     │             │                     │
├───────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ lstm_1 (LSTM)         │ (None, 50)          │      10,400 │ time_series_input[… │
├───────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ fixed_input           │ (None, 2)           │           0 │ -                   │
│ (InputLayer)          │                     │             │                     │
├───────────────────────┼─────────────────────┼─────────────┼─────────────────────┤
│ concatenate_1         │ (None, 52)          │           0 │ lstm_1[0][0], 

 Total params: 10,453 (40.83 KB)
 Total params: 10,453 (40.83 KB)


 Trainable params: 10,453 (40.83 KB)
 Trainable params: 10,453 (40.83 KB)


 Non-trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)


In [ ]:
# Entraîner le modèle avec les données
model.fit([X_time_series, X_fixed], targets, epochs=100, batch_size=1)

In [42]:
# Padding des séries temporelles pour qu'elles aient toutes la même longueur
padded_series_test = [
    [0] * (max_length - len(player['past'])) + player['past']
    for player in test_dict.values()
]
ages = np.array([player['age'] for player in test_dict.values()])
heights = np.array([player['height'] for player in test_dict.values()])

# Transformer les données en matrices (entrée pour le LSTM)
X_time_series_test = np.array(padded_series_test).reshape(len(test_dict), max_length, 1)
X_fixed_test = np.column_stack((ages, heights))  # Variables fixes

# Faire des prédictions après l'entraînement
predictions = model.predict([X_time_series_test, X_fixed_test])
print(X_time_series_test[1])
print("Prediction:", predictions[1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


[[17]
 [17]
 [11]
 [11]
 [ 4]
 [12]
 [ 7]
 [14]
 [11]
 [ 7]]
Prediction: [10.150367]
[[17]
 [17]
 [11]
 [11]
 [ 4]
 [12]
 [ 7]
 [14]
 [11]
 [ 7]]
Prediction: [10.150367]
